# Code to get AVERAGE SPEED / ACCELERATION

In [1]:
# Client code
import requests

# Base URL for the FastAPI app
base_url = "http://localhost:8000"
num_players = 3  # Number of players
run_id = "run_001"  # Run ID

# Construct the URL with path parameters
url = f"{base_url}/get-team-average-stats/{num_players}/{run_id}"

# Make the GET request
response = requests.get(url)

# Process the response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code, response.text)

{'run_id': 'run_001', 'team_average_speed': 1.4576678577631912, 'team_average_acceleration': 0.18341438240936792}


# Code to get AVERAGE TEMPERATURE / HEART Rate

In [2]:
import requests

# API endpoint configuration
BASE_URL = "http://127.0.0.1:8000"
ENDPOINT = "/get-team-kafka-stats/{num_players}/{run_id}"

# Parameters
num_players = 3
run_id = "run_001"

# Build the full URL
url = BASE_URL + ENDPOINT.format(num_players=num_players, run_id=run_id)

try:
    # Make the GET request
    response = requests.get(url)

    # Check the response status code
    if response.status_code == 200:
        data = response.json()
        print(data)  # Print the response JSON
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Error: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


{'run_id': 'run_001', 'team_average_heart_rate': 125.66666666666667, 'team_average_temperature': 38.3}


# Code to get ALARM for HEART RATE

In [5]:
import requests

# API endpoint configuration
BASE_URL = "http://127.0.0.1:8000"
ENDPOINT = "/analyze-heart-rate/{num_players}/{run_id}"

# Parameters
num_players = 3
run_id = "run_001"

# Build the full URL
url = BASE_URL + ENDPOINT.format(num_players=num_players, run_id=run_id)

try:
    # Make the GET request
    response = requests.get(url)

    # Check the response status code
    if response.status_code == 200:
        data = response.json()
        print(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Error: {response.text}")
except Exception as e:
    print(f"An error occurred: {e}")


{'run_id': 'run_001', 'analyzed_data': [{'device_id': 'player_heart_rate_1', 'heart_rate': 88, 'alarm_type': 'normal', 'timestamp': '2024-12-10T17:40:45.523639'}, {'device_id': 'player_heart_rate_2', 'heart_rate': 123, 'alarm_type': 'normal', 'timestamp': '2024-12-10T17:40:45.259818'}, {'device_id': 'player_heart_rate_3', 'heart_rate': 116, 'alarm_type': 'normal', 'timestamp': '2024-12-10T17:40:45.570671'}]}
